Lower-bounding f_n and h_n using a mixed integer programming framework (with the Branch-and-cut algorithm).

In [1]:
%run constants.ipynb
%run gen_h.ipynb
import gurobipy as gp
from gurobipy import GRB
from itertools import repeat
import re

In [2]:
def var_name(*ineqs, prefix=''):
    '''
    Returns the variable name for the MIP.
    '''
    return prefix + f'{str(ineqs)}'

In [3]:
def extract_ineq(var):
    '''
    Extracts the inequality from the given variable's name.
    (For interpretability, we associate each indicator variable's name 
    with the inequality(s) they represent)
    '''
    match = re.findall(r'\(([^()]+)\)', var.varName)
    return sympify(match[0]) if match else None

In [4]:
def extract_ineqs(variables):
    '''
    Extracts inequalities from multiple variables, 
    based on their values (i.e. 1 = true, 0 = false).
    '''
    ineqs = []
    for v in variables:
        e = extract_ineq(v)
        if v.x == 1 and e is not None:  # indicator is true
            ineqs.append(e)
        elif v.x == 0:  # indicator is false
            ineqs.append(Not(And(e)))
    return ineqs

In [5]:
def to_gp_linexpr(sp_expr, var_map):
    '''
    Converts a sympy expression to a gurobipy linear expression,
    using the provided (sp : gp) variable mapping.
    '''
    gp_expr = gp.LinExpr()

    if isinstance(sp_expr, Add):
        for term in sp_expr.args:
            gp_expr.add(to_gp_linexpr(term, var_map))
    elif sp_expr.is_Mul:
        coeff = 1
        var = None
        for factor in sp_expr.args:
            if factor.is_number:  # sp coefficient
                coeff *= float(factor)
            elif factor in var_map:  # sp variable
                var = var_map[factor]
        if var is not None:
            gp_expr.addTerms(coeff, var)
        else:  # constant
            gp_expr.addConstant(coeff)
    elif sp_expr in var_map:  # single variable expression
        gp_expr.addTerms(1.0, var_map[sp_expr])
    elif sp_expr.is_number:   # single constant
        gp_expr.addConstant(float(sp_expr))

    return gp_expr

In [8]:
EPS = 1
def unify_ineq(ineq, eps, scale_by=1):
    '''
    Converts the given inequality to the form expr >= 0.
    '''
    lhs, rhs, op = ineq.lhs, ineq.rhs, ineq.rel_op
    # Check the inequality type and unify the format
    if op == '>=':
        # lhs >= rhs -> lhs - rhs >= 0
        return Rel(scale_by * (lhs - rhs), 0, '>=')
    elif op == '>':
        # lhs > rhs -> lhs - rhs > 0 => lhs - rhs >= eps
        return Rel(scale_by * (lhs - rhs) - eps, 0, '>=')
    elif op == '<=':
        # lhs <= rhs -> lhs - rhs <= 0 -> rhs - lhs >= 0
        return Rel(scale_by * (rhs - lhs), 0, '>=')
    elif op == '<':
        # lhs < rhs -> rhs > lhs -> rhs - lhs > 0 -> rhs - lhs >= eps
        return Rel(scale_by * (rhs - lhs - eps), 0, '>=')
    else:
        raise ValueError(f"Unhandled inequality type: {op}")

In [9]:
def to_gp_ineq(ineq, var_map, scale_by=1, eps=EPS):
    '''
    Like `to_gp_linexpr`, but converts a sympy inequality to a gurobipy inequality 
    (of the form >= 0).
    '''
    return (to_gp_linexpr(unify_ineq(ineq, eps, scale_by=scale_by).lhs, var_map) >= 0)

In [10]:
def add_ind_constr(ineq, model=None, name='ind', var_map=None, eps=EPS, scale_by=1):
    ''' 
    Adds indicator constraint to the model for a single inequality. 
    '''
    lhs = to_gp_linexpr(unify_ineq(ineq, eps, scale_by=scale_by).lhs, var_map)  # to the form >= 0
    v = model.addVar(vtype=GRB.BINARY, name=name)
    model.addConstr((v == 1) >> (lhs >= 0))
    model.addConstr((v == 0) >> (lhs <= -eps))  # lhs < 0
    return v

In [11]:
def add_ind_constrs(*ineqs, model=None, var_map=None, eps=EPS, scale_by=1):
    ''' 
    Adds variables and constraints to the model for a compound indicator 
    (i.e. may represent the and of multiple inequalities). 
    '''
    if len(ineqs) == 1:
        return (add_ind_constr(ineqs[0], 
                               name=var_name(ineqs[0], prefix='single_ind'), 
                               var_map=var_map,
                               model=model,
                               scale_by=scale_by,
                               eps=eps),)
    
    I = model.addVar(vtype=GRB.BINARY, 
                     name=var_name(*ineqs, prefix='main_ind'))
    component_vars = [add_ind_constr(i, 
                                     name=var_name(i, prefix='component_ind'), 
                                     var_map=var_map,
                                     model=model,
                                     scale_by=scale_by,
                                     eps=eps) for i in ineqs]
    model.addConstr(I == gp.and_(component_vars), 
                    name=var_name(*ineqs, prefix='andconstr'))
    return I, component_vars

In [12]:
def analyse_result(model, relax=False):
    '''
    Analyses the status of the model.
    If the model is feasible and `relax` is True, then re-optimizes the model with relaxed bounds.
    Note that if the relaxation is infeasible, then the exact model must too be infeasible
    '''
    if model.status == GRB.OPTIMAL:
        print('Optimal solution found:')
        for v in model.getVars():
            print(f"{v.varName}: {v.x}")
        print(f'Objective value:', model.objVal)
        print(f'MIPGap = |ObjBound - ObjVal|/|ObjVal|:', model.MIPGap)
    elif model.status == GRB.UNBOUNDED:
        print('The model cannot be solved because it is unbounded')
    elif model.status != GRB.INF_OR_UNBD and model.status != GRB.INFEASIBLE:
        print('Optimization was stopped with status %d' % model.status)
    else:
        # Relax the constraints and try to make the model feasible
        print('The model is infeasible')
        if relax:
            print('Relaxing the bounds')
            model.feasRelaxS(1, False, False, True)
            model.optimize()
            analyse_result(model, relax=relax)

In [22]:
def delta_mip(n, output_file=False, analyse=True, relax=False):
    '''
    Runs MIP model to minimize f_n - thresh(n).
    If `output_file` is True, writes the model as a '.lp' file.
    If `analyse` is True, analyses the optimization results.
    '''
    # uncomment the following to collect the regions cut out (i.e. excluded) by the model
#     failed_regions = []
    
#     def extract_region(model):
#         region = []
#         variables = model.getVars()
#         sol = model.cbGetSolution(variables)
#         for v, val in zip(variables, sol):
#             # only look at the individual indicators
#             if not 'single_ind' in v.varName and not 'component_ind' in v.varName:
#                 continue
#             e = extract_ineq(v)
#             if val == 1 and e is not None:
#                 region.append(e[0])
#             elif val == 0:
#                 # indicator is false
#                 # guaranteed single inequality
#                 region.append(Not(e[0]))
#         return region
    
#     def pos_f_callback(model, where):
#         if where == GRB.Callback.MIPSOL:
#             # Capture node information
#             try:
#                 obj = model.cbGet(GRB.Callback.MIPSOL_OBJ)
#                 print("Current objective:", obj)
#                 if obj > 0:  # Node is infeasible
#                     failed_regions.append(extract_region(model))
#             except gp.GurobiError as e:
#                 print(f"Gurobi error during callback: {e}")
#             except Exception as e:
#                 print(f"Unexpected error during callback: {e}")
    
    pos_h = read(POS_H_CACHE, min_n=1, max_n=n)
    neg_h = read(NEG_H_CACHE, min_n=1, max_n=n)
    num_pos_per_n = [len(pos_h[i][1]) for i in range(1,n+1)]
    num_neg_per_n = [len(neg_h[i][1]) for i in range(1,n+1)]
    num_pos = sum(num_pos_per_n)
    num_neg = sum(num_neg_per_n)
    
    pos_coefs = concat([repeat(j, k) for j,k in zip([coef(i) for i in range(1,n+1)], num_pos_per_n)])
    neg_coefs = concat([repeat(j, k) for j,k in zip([-coef(i) for i in range(1,n+1)], num_neg_per_n)])
    const = sum([coef(i)*(pos_h[i][0] - neg_h[i][0]) for i in range(1,n+1)])
    
    all_pos_comp_inds = concat([pos_h[i][1] for i in range(1,n+1)])
    all_neg_comp_inds = concat([neg_h[i][1] for i in range(1,n+1)])
    
    model = gp.Model(F_MODEL_NAME(n), env=gp.Env())
    model.Params.OutputFlag = 1
    model.Params.LogToConsole = 1
#     model.Params.MIPFocus = 1
    
    gx = model.addVar(lb=0, name='x')
    gy = model.addVar(lb=0, name='y')
    gz = model.addVar(lb=0, name='z')
    sp_to_gp = {x: gx, y: gy, z: gz}
    
    pos_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp, eps=EPS) \
                for i in all_pos_comp_inds]
    neg_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp, eps=EPS) \
                for i in all_neg_comp_inds]

    obj = gp.quicksum([pos_coefs[i]*pos_inds[i][0] for i in range(num_pos)]) + \
        gp.quicksum([neg_coefs[i]*neg_inds[i][0] for i in range(num_neg)]) + const - thresh(n)
    
    # constraints for V
    for i, ineq in enumerate(V):
        model.addConstr(to_gp_ineq(ineq, sp_to_gp, eps=EPS), name=f'V{i}')
        
    delta_constr = model.addConstr(obj <= 0, name='non_pos_delta')
    
#     model.optimize(pos_f_callback)
    model.setObjective(obj, GRB.MINIMIZE)
#     model.setObjective(0, GRB.MAXIMIZE)
    model.optimize()
    
    if analyse:
        analyse_result(model, relax=relax)
    
    if output_file:
        model.write(F_MIP_OUTPUT(n))
    
    return ((model.status == GRB.OPTIMAL and \
            model.MIPGap == 0 and \
            sub(V, {x:gx.x, y:gy.x, z:gz.x}) == 1 and \
            eval_delta(n, gx.x, gy.x, gz.x) < 0), # sanity check using the x,y,z values from the model
            model)

In [ ]:
status, model = delta_mip(5, analyse=True, relax=False)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.6.0 23G93)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 10690 columns and 3097 nonzeros
Model fingerprint: 0x5b053e66
Model has 18566 general constraints
Variable types: 3 continuous, 10687 integer (10687 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+00]
  Objective range  [1e-04, 2e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-02, 1e+00]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 3e+01]
Presolve added 27586 rows and 23637 columns
Presolve time: 2.11s
Presolved: 27590 rows, 34327 columns, 84178 nonzeros
Presolved model has 15758 SOS constraint(s)
Variable types: 15761 continuous, 18566 integer (18566 binary)

Root relaxation: objective -7.204733e-01, 16732 iterations, 0.08 seconds (0.05 wo

 55034    33 infeasible 1357               -   -0.71379      -   6.9  869s
 55884    34   -0.52616 1415 2728          -   -0.71379      -   6.8  895s
 56499    32 infeasible 1456               -   -0.71379      -   6.8  921s
 57355    33   -0.47215 1513 2638          -   -0.71379      -   6.8  949s
 58200    34 infeasible 1569               -   -0.71379      -   6.8  977s
 59029    35   -0.44270 1627 2550          -   -0.71379      -   6.8 1002s
 59822    35 infeasible 1683               -   -0.71379      -   6.9 1033s
 60508    34   -0.43112 1733 2465          -   -0.71379      -   6.9 1059s
 61311    35   -0.43112 1791 2408          -   -0.71379      -   7.2 1090s
 62110    36   -0.43112 1849 2342          -   -0.71379      -   7.2 1121s
 62699    33   -0.43112 1893 2292          -   -0.71379      -   7.2 1161s
 63528    39   -0.43112 1953 2230          -   -0.71379      -   7.1 1192s
 64348    35   -0.43112 2013 2165          -   -0.71379      -   7.2 1224s
 65176    39   -0.43112 2

 141630    58 infeasible 3565               -   -0.71379      -  25.6 4845s
 142449    57 infeasible 3621               -   -0.71379      -  25.7 4893s
 143312    64 infeasible 3679               -   -0.71379      -  25.7 4939s
 144089    70   -0.71379   76 4039          -   -0.71379      -  26.4 4986s
 144959    69   -0.71173  129 4007          -   -0.71379      -  26.6 5030s
*145405    69            4230      -0.0135545   -0.71379  5166%  26.5 5030s
 145924  1001   -0.70414  196 3972   -0.01355   -0.71379  5166%  26.5 5072s
H147094  1001                      -0.0139403   -0.71379  5020%  27.4 5072s
 149118  2722   -0.40656  459 3769   -0.01394   -0.71379  5020%  27.3 5091s
 153027  4149   -0.37402  965 3221   -0.01394   -0.71379  5020%  27.1 5112s
 155636  5487   -0.34586 1471 2687   -0.01394   -0.71379  5020%  26.8 5125s
 157928  6693   -0.28066 1748 2422   -0.01394   -0.71379  5020%  26.6 5139s
 159616  7619   -0.26896 2177 2019   -0.01394   -0.71379  5020%  26.4 5150s
 161372  777

 237604 40906   -0.22060 3213  858   -0.02487   -0.71379  2770%  19.8 6711s
 238142 41432   -0.22047 3214  858   -0.02487   -0.71379  2770%  19.8 6725s
 238924 42915   -0.21546 3241  839   -0.02487   -0.71379  2770%  19.7 6734s
 241705 43501   -0.17199 3268  817   -0.02487   -0.71379  2770%  19.5 6844s
 242405 44886   -0.17199 3269  816   -0.02487   -0.71379  2770%  19.5 6858s
 244644 45476   -0.16337 3296  792   -0.02487   -0.71379  2770%  19.3 6876s
 245600 46566   -0.15231 3323  772   -0.02487   -0.71379  2770%  19.2 7911s
 247694 47172   -0.14756 3350  750   -0.02487   -0.71379  2770%  19.1 7922s
 249224 47174   -0.14756 3377  722   -0.02487   -0.71379  2770%  19.1 7943s
 249238 48441   -0.14756 3378  721   -0.02487   -0.71379  2770%  19.1 7955s
 251111 48995   -0.69720  208 3767   -0.02487   -0.71379  2770%  18.9 7960s
 252369 50334   -0.69642  268 3709   -0.02487   -0.71379  2770%  18.9 7977s
 254664 51364   -0.65977  705 3252   -0.02487   -0.71379  2770%  18.7 7986s
 256432 5271

 520310 150208 infeasible  886        -0.02487   -0.65167  2520%  14.7 9896s
 522714 149645 infeasible 1340        -0.02487   -0.64859  2508%  14.7 9900s
 525317 149155 infeasible 1300        -0.02487   -0.64563  2496%  14.7 9906s
 527927 148696 infeasible  964        -0.02487   -0.64203  2481%  14.7 9910s
 531790 149794   -0.61489  649 3483   -0.02487   -0.64203  2481%  14.6 9916s
 534702 150414   -0.58480 1217 2641   -0.02487   -0.64203  2481%  14.6 9920s
 538847 152569   -0.49426 1965 1873   -0.02487   -0.64203  2481%  14.5 9925s
 543113 155363   -0.37094 2668 1263   -0.02487   -0.64203  2481%  14.4 9930s
 547228 157947   -0.12287 3339  705   -0.02487   -0.64203  2481%  14.3 9935s
 550959 159670   -0.57258 2109 1834   -0.02487   -0.64048  2475%  14.2 9940s
 554423 161076   -0.43356 2764 1249   -0.02487   -0.64048  2475%  14.2 9945s
 558065 162817   -0.12981 3431  700   -0.02487   -0.64048  2475%  14.1 9950s
 561771 164601 infeasible  828        -0.02487   -0.63675  2460%  14.0 9955s

...

 2144408 893709   -0.52307 1828 2167   -0.02487   -0.62235  2402%   8.9 11985s
 2147232 894603 infeasible 1966        -0.02487   -0.62235  2402%   8.9 11990s
 2151185 896740   -0.26458 2141 1923   -0.02487   -0.62235  2402%   8.9 11996s
 2153917 897702   -0.26098 2266 1807   -0.02487   -0.62235  2402%   8.9 12000s
 2157483 898835   -0.25262 2425 1672   -0.02487   -0.62235  2402%   8.9 12005s
 2161058 899963   -0.24028 2587 1510   -0.02487   -0.62235  2402%   9.0 12010s
 2163998 901592 infeasible 2704        -0.02487   -0.62235  2402%   9.0 12015s
 2166826 902568   -0.23359 2831 1269   -0.02487   -0.62235  2402%   8.9 12020s
 2169882 904649 infeasible 2941        -0.02487   -0.62235  2402%   8.9 12025s
 2172969 906661   -0.21237 3058 1081   -0.02487   -0.62235  2402%   8.9 12030s
 2175050 907987   -0.17495 3639  386   -0.02487   -0.62235  2402%   8.9 12035s
 2178750 909972   -0.19295 3266  912   -0.02487   -0.62222  2402%   8.9 12041s
 2179688 911639 infeasible  809        -0.02487   -0

 2538446 1071645   -0.08801 3768  374   -0.02487   -0.62106  2397%   8.7 12505s
 2541680 1072437   -0.06268 3948  260   -0.02487   -0.62106  2397%   8.7 12510s
 2545209 1074016   -0.62055  862 3038   -0.02487   -0.62068  2396%   8.7 12515s
 2549180 1076144   -0.60962 1571 2318   -0.02487   -0.62068  2396%   8.7 12520s
 2553056 1078217   -0.31512 2264 1724   -0.02487   -0.62068  2396%   8.7 12525s
 2556787 1080154   -0.28799 2947 1101   -0.02487   -0.62068  2396%   8.7 12530s
 2560999 1081348 infeasible  939        -0.02487   -0.62068  2396%   8.7 12535s
 2564311 1082752   -0.40874 1486 2449   -0.02487   -0.62068  2396%   8.7 12540s
 2568774 1084545   -0.30278 2376 1634   -0.02487   -0.62068  2396%   8.7 12545s
 2572392 1085888   -0.28130 3094  960   -0.02487   -0.62068  2396%   8.7 12550s
 2576828 1087542 infeasible 1010        -0.02487   -0.62068  2396%   8.7 12555s
 2580658 1089885   -0.58930 1301 2639   -0.02487   -0.62068  2396%   8.7 12560s
 2585816 1093282   -0.29468 2161 1867   

 2949080 1230889   -0.35139 2018 1852   -0.02487   -0.61914  2389%   8.5 13020s
 2953398 1232404 infeasible 2236        -0.02487   -0.61914  2389%   8.5 13025s
 2957075 1234477   -0.33981 2414 1507   -0.02487   -0.61914  2389%   8.5 13030s
 2961668 1236493   -0.32966 2623 1320   -0.02487   -0.61914  2389%   8.5 13035s
 2965407 1238382   -0.31512 2789 1192   -0.02487   -0.61914  2389%   8.5 13040s
 2969787 1240012 infeasible 3019        -0.02487   -0.61914  2389%   8.4 13045s
 2974080 1241758   -0.26523 3254  788   -0.02487   -0.61914  2389%   8.4 13051s
 2977913 1243370   -0.23050 3442  622   -0.02487   -0.61914  2389%   8.4 13055s
 2981659 1245164   -0.20697 3629  462   -0.02487   -0.61914  2389%   8.4 13060s
 2985923 1246263 infeasible 1120        -0.02487   -0.61875  2388%   8.4 13066s
 2989898 1248281   -0.32734 1185 2686   -0.02487   -0.61875  2388%   8.4 13071s
 2993748 1250004   -0.25365 1897 2079   -0.02487   -0.61875  2388%   8.4 13075s
 2997369 1251421   -0.24246 2603 1357   

 3307751 1393779 infeasible  964        -0.02487   -0.61656  2379%   8.3 13555s
 3310277 1394159   -0.58017 1736 2159   -0.02487   -0.61656  2379%   8.3 13560s
 3313629 1394588   -0.31191 2476 1526   -0.02487   -0.61656  2379%   8.3 13565s
 3317289 1396009   -0.23835 3225  898   -0.02487   -0.61656  2379%   8.2 13571s
 3320843 1396581 infeasible 1089        -0.02487   -0.61644  2378%   8.2 13576s
 3324342 1397456   -0.61438 1360 2528   -0.02487   -0.61631  2378%   8.2 13582s
 3326233 1398003   -0.60255 1734 2188   -0.02487   -0.61631  2378%   8.2 13585s
 3329145 1399113   -0.32271 2293 1697   -0.02487   -0.61631  2378%   8.2 13591s
 3332956 1400527   -0.26973 3021 1090   -0.02487   -0.61631  2378%   8.2 13596s
 3335851 1401720 infeasible 1164        -0.02487   -0.61631  2378%   8.2 13600s
 3339366 1402460 infeasible  481        -0.02487   -0.61618  2377%   8.2 13605s
 3342444 1404396   -0.37055 1524 2280   -0.02487   -0.61618  2377%   8.2 13612s
 3345683 1406593   -0.31860 2035 1849   

 3701999 1574426 infeasible 1035        -0.02487   -0.61412  2369%   8.3 14071s
 3705088 1575026   -0.58441 1195 2770   -0.02487   -0.61412  2369%   8.3 14075s
 3709030 1575816 infeasible 1405        -0.02487   -0.61412  2369%   8.3 14080s
 3712770 1575701   -0.55381 1628 2309   -0.02487   -0.61412  2369%   8.3 14086s
 3715489 1576152   -0.54956 1782 2169   -0.02487   -0.61412  2369%   8.3 14090s
 3719206 1577282 infeasible 1983        -0.02487   -0.61412  2369%   8.3 14095s
 3723473 1579806   -0.49864 2182 1815   -0.02487   -0.61412  2369%   8.3 14100s
 3728775 1583506   -0.28735 2411 1624   -0.02487   -0.61412  2369%   8.3 14105s
 3732685 1585886 infeasible 2587        -0.02487   -0.61412  2369%   8.3 14110s


In [18]:
eval_delta(5, 
           model.getVarByName('gx').x, 
           model.getVarByName('gy').x, 
           model.getVarByName('gz').x)

-0.1

# LB on h_n(x,y,z) - h_n(y,x,z)

In [24]:
def h_mip(n, output_file=False, analyse=True, relax=False):
    '''
    Like `f_mip`, but runs MIP model to minimize h_n(x,y,z) - h_n(y,x,z).
    '''
    pos_h = read(POS_H_CACHE, min_n=1, max_n=n)
    neg_h = read(NEG_H_CACHE, min_n=1, max_n=n)
    
    all_pos_comp_inds = concat([pos_h[i][1] for i in range(1,n+1)])
    all_neg_comp_inds = concat([neg_h[i][1] for i in range(1,n+1)])
    
    model = gp.Model(DH_MODEL_NAME(n), env=gp.Env())
    model.Params.OutputFlag = 1
    model.Params.LogToConsole = 1
#     model.Params.MIPFocus = 1
    
    gx = model.addVar(lb=0, name='x')
    gy = model.addVar(lb=0, name='y')
    gz = model.addVar(lb=0, name='z')
    sp_to_gp = {x: gx, y: gy, z: gz}
    
    pos_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp, eps=EPS) for i in pos_h[n][1]]
    neg_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp, eps=EPS) for i in neg_h[n][1]]
    const = pos_h[n][0] - neg_h[n][0]
    
    obj = coef(n) * (gp.quicksum([i[0] for i in pos_inds]) - gp.quicksum([i[0] for i in neg_inds]) + const)
    
    model.setObjective(obj, GRB.MINIMIZE)
    for i, ineq in enumerate(V):
        model.addConstr(to_gp_ineq(ineq, sp_to_gp, eps=EPS), name=f'V{i}')
    
    model.optimize()
    
    if analyse:
        analyse_result(model, relax=relax)
    
    if output_file:
        model.write(DH_MIP_OUTPUT(n))
    
    return (model.status == GRB.OPTIMAL and model.MIPGap == 0), model

In [25]:
h_mip(4, analyse=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.6.0 23G93)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 1465 columns and 4 nonzeros
Model fingerprint: 0x278bf0de
Model has 2524 general constraints
Variable types: 3 continuous, 1462 integer (1462 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-04, 8e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 2e+01]
Presolve added 3742 rows and 3186 columns
Presolve time: 0.07s
Presolved: 3745 rows, 4651 columns, 10747 nonzeros
Presolved model has 2124 SOS constraint(s)
Variable types: 2127 continuous, 2524 integer (2524 binary)

Root relaxation: objective -1.180556e-01, 2205 iterations, 0.01 seconds (0.01 work units)

    N

(True,
 <gurobi.Model MIP instance h4(x,y,z)-h4(y,y,z): 3 constrs, 1465 vars, Parameter changes: Username=(user-defined)>)

In [26]:
eval_dh(4,8,9,8)

Fraction(-7, 648)

# f(x, x, z) - f((x+y)/2, (x+y)/2, z) where x+y+z = FIXED_SUM

In [ ]:
n=5
pos_h = read(POS_H_CACHE, min_n=1, max_n=n)
neg_h = read(NEG_H_CACHE, min_n=1, max_n=n)

In [13]:
num_pos_per_n = [len(pos_h[i][1]) for i in range(1,n+1)]
num_neg_per_n = [len(neg_h[i][1]) for i in range(1,n+1)]
num_pos = sum(num_pos_per_n)
num_neg = sum(num_neg_per_n)

pos_coefs = concat([repeat(j, k) for j,k in zip([coef(i) for i in range(1,n+1)], num_pos_per_n)])
neg_coefs = concat([repeat(j, k) for j,k in zip([-coef(i) for i in range(1,n+1)], num_neg_per_n)])
const = sum([coef(i)*(neg_h[i][0] - neg_h[i][0]) for i in range(1,n+1)])

/var/folders/tk/2zq7clf94gx3rdh5slf6rhc00000gn/T/ipykernel_1082/3349906455.py:14: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  if not isinstance(args, collections.Hashable):


In [14]:
all_pos_comp_inds = concat([pos_h[i][1] for i in range(1,n+1)])
all_neg_comp_inds = concat([neg_h[i][1] for i in range(1,n+1)])

## Real version

In [48]:
EPS = 1
SCALE=10000

In [49]:
model = gp.Model(f"dh'{n}_model", env=gp.Env())
model.Params.OutputFlag = 1
model.Params.LogToConsole = 1

gx = model.addVar(lb=EPS, ub=SCALE, name='x')
gy = model.addVar(lb=EPS, ub=SCALE, name='y')
gz = model.addVar(lb=Z_LB * SCALE, ub=SCALE, name='z')
# dummy variable for the x in h(x,x,z), since this x'=(1-z)/2=(x+y)/2
sp_to_gp = {x: gx, y: gy, z: gz}

# constraints for V
model.addConstr(gx + gy + gz == SCALE, name=f'x+y+z=1')
model.addConstr(gx <= gy - EPS, name=f'x<y')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02
Set parameter MIPFocus to value 1


<gurobi.Constr *Awaiting Model Update*>

In [50]:
pos_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp, eps=EPS) \
                for i in all_pos_comp_inds]
neg_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp, eps=EPS) \
            for i in all_neg_comp_inds]

In [51]:
obj = gp.quicksum([pos_coefs[i]*pos_inds[i][0] for i in range(num_pos)]) + \
        gp.quicksum([neg_coefs[i]*neg_inds[i][0] for i in range(num_neg)]) + const - thresh(n)
    
model.setObjective(obj, GRB.MINIMIZE)

In [52]:
delta_constr = model.addConstr(obj <= 0, name='non_pos_delta')

In [53]:
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 5856 columns and 1895 nonzeros
Model fingerprint: 0x9a011c5a
Model has 10156 general constraints
Variable types: 0 continuous, 5856 integer (5853 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+00]
  Objective range  [1e-04, 3e-02]
  Bounds range     [1e+00, 1e+04]
  RHS range        [3e-02, 1e+04]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 3e+01]
Presolve added 10440 rows and 0 columns
Presolve removed 0 rows and 87 columns
Presolve time: 0.48s
Presolved: 10443 rows, 5769 columns, 39265 nonzeros
Variable types: 0 continuous, 5769 integer (5766 binary)

Root relaxation: objective -1.976494e-01, 6407 iterations, 0.59 seconds (1.35 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth

In [42]:
analyse_result(model)

Optimal solution found:
x: 308.0
y: 346.0
z: 346.0
single_ind(3*x <= y,): 0.0
single_ind(2*x <= z,): 0.0
single_ind(2*x <= y,): 0.0
single_ind(3*x <= z,): -0.0
single_ind(y < z,): 0.0
single_ind(x <= -y + z,): 0.0
single_ind(x <= y - z,): -0.0
single_ind(z < y,): 0.0
single_ind(7*x <= y,): -0.0
main_ind(4*x <= z, 3*x < y): -0.0
component_ind(4*x <= z,): 0.0
component_ind(3*x < y,): 0.0
single_ind(5*x <= y,): -0.0
single_ind(6*x <= z,): -0.0
main_ind(5*x <= 3*y, y < 3*x): 0.0
component_ind(5*x <= 3*y,): 0.0
component_ind(y < 3*x,): 1.0
main_ind(3*x <= y + z, y < 3*x): 0.0
component_ind(3*x <= y + z,): 0.0
component_ind(y < 3*x,): 1.0
main_ind(2*x <= y, y - z < x): 0.0
component_ind(2*x <= y,): 0.0
component_ind(y - z < x,): 1.0
single_ind(x <= -y + z,): -0.0
main_ind(3*x <= y + z, z < 2*x): 0.0
component_ind(3*x <= y + z,): 0.0
component_ind(z < 2*x,): 1.0
single_ind(z < 2*x,): 1.0
single_ind(3*x <= y - z,): 0.0
single_ind(x < y - z,): -0.0
single_ind(6*x <= y,): 0.0
main_ind(5*x <= z, 

In [ ]:
%run constants.ipynb
%run gen_h.ipynb
%run verification.ipynb

In [340]:
delta_dash(7, 332,334,334)

-0.03958261888431641

In [ ]:
delta_dash(12,332,334,334)

In [60]:
delta_R(4,26,27,4,dh=dh_R_thm6)

-0.05

In [ ]:
delta_R(4,26,27,4)